In [5]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


In [20]:
for target in train_targets:
    for n,value in enumerate(target):
        if value < .1:
            target[n] = .1/len(target)
        else:
            target[n] = .9
train_targets[1]

array([  7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,   7.51879699e-04,   7.51879699e-04,
         7.51879699e-04,

In [140]:
train_xception.shape

(6680, 7, 7, 2048)

In [1]:

bottleneck_features_x = np.load('bottleneck_features/DogXceptionData.npz')
train_xception = bottleneck_features_x['train']
valid_xception = bottleneck_features_x['valid']
test_xception = bottleneck_features_x['test']

In [146]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, BatchNormalization, Activation
from keras.models import Sequential
xception_model = Sequential()
xception_model.add(Conv2D(filters=512, kernel_size=1, padding='same', activation='relu', input_shape=(7,7,2048)))
xception_model.add(Conv2D(filters = 2048, kernel_size =3,padding='same', activation = 'relu'))
xception_model.add(Dropout(.3))
xception_model.add(GlobalAveragePooling2D())
xception_model.add(BatchNormalization())
xception_model.add(Dense(256, activation='relu'))
xception_model.add(BatchNormalization())
xception_model.add(Dropout(.3))
#
xception_model.add(Dense(133, activation='softmax'))
#xception_model.add(Activation('softmax'))

xception_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 7, 7, 512)         1049088   
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 7, 7, 2048)        9439232   
_________________________________________________________________
dropout_49 (Dropout)         (None, 7, 7, 2048)        0         
_________________________________________________________________
global_average_pooling2d_72  (None, 2048)              0         
_________________________________________________________________
batch_normalization_141 (Bat (None, 2048)              8192      
_________________________________________________________________
dense_143 (Dense)            (None, 256)               524544    
_________________________________________________________________
batch_normalization_142 (Bat (None, 256)               1024      
__________

In [147]:
xception_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [148]:
from keras.callbacks import ModelCheckpoint 
checkpointer = ModelCheckpoint(filepath='saved_models/weights.bagging2.hdf5', 
                               verbose=1, save_best_only=True)

xception_model.fit(train_xception, train_targets, 
          validation_data=(valid_xception, valid_targets),
          epochs=20, batch_size=100, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/20
6680/6680 [==============================] - 692s 104ms/step - loss: 2.0860 - acc: 0.6626 - val_loss: 1.8607 - val_acc: 0.5593
Epoch 2/20
6680/6680 [==============================] - 658s 99ms/step - loss: 1.4448 - acc: 0.8262 - val_loss: 0.9461 - val_acc: 0.7509
Epoch 3/20
6680/6680 [==============================] - 635s 95ms/step - loss: 1.3186 - acc: 0.8711 - val_loss: 0.7170 - val_acc: 0.8036
Epoch 4/20
6680/6680 [==============================] - 631s 94ms/step - loss: 1.2289 - acc: 0.9066 - val_loss: 0.7576 - val_acc: 0.8060
Epoch 5/20
6680/6680 [==============================] - 627s 94ms/step - loss: 1.1487 - acc: 0.9341 - val_loss: 0.7011 - val_acc: 0.8287
Epoch 6/20
6680/6680 [==============================] - 628s 94ms/step - loss: 1.0958 - acc: 0.9530 - val_loss: 0.7267 - val_acc: 0.8180
Epoch 7/20
6680/6680 [==============================] - 623s 93ms/step - loss: 1.0579 - acc: 0.9663 - val_loss: 0.7200 - val_acc: 

In [149]:
xception_predictions = [np.argmax(xception_model.predict(np.expand_dims(feature, axis=0))) for feature in test_xception]

# report test accuracy
test_accuracy = 100*np.sum(np.array(xception_predictions)==np.argmax(test_targets, axis=1))/len(xception_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 84.5694%


In [26]:
xception_model.save_weights('saved_models/xceptbag.hdf5')

In [28]:
xception_model.load_weights('saved_models/weights.bagging.hdf5')

In [59]:
from random import randint
Xcept = [Sequential() for i in range (20)]


def buildModel (model):
    model.add(GlobalAveragePooling2D(input_shape=train_xception.shape[1:]))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(.3))
    model.add(BatchNormalization())
    model.add(Dense(133, activation='linear'))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    indxes = np.random.randint(0,high= train_xception.shape[0], size = train_xception.shape[0], dtype = 'int32')
    train = np.array([train_xception[ind] for ind in indexes])
    targ = np.array([train_targets[ind]for ind in indexes])
                     
    model.fit(train, targ, 
          validation_data=(valid_xception, valid_targets),
          epochs=10, batch_size=100, verbose=1)
for model in Xcept:
    buildModel(model)

    

Train on 6680 samples, validate on 835 samples
Epoch 1/10
6680/6680 [==============================] - 7s 1ms/step - loss: 2.0934 - acc: 0.6771 - val_loss: 1.9025 - val_acc: 0.8096
Epoch 2/10
6680/6680 [==============================] - 4s 655us/step - loss: 1.2493 - acc: 0.9027 - val_loss: 1.5058 - val_acc: 0.8407
Epoch 3/10
6680/6680 [==============================] - 4s 641us/step - loss: 1.1318 - acc: 0.9484 - val_loss: 1.1952 - val_acc: 0.8347
Epoch 4/10
6680/6680 [==============================] - 4s 641us/step - loss: 1.0739 - acc: 0.9665 - val_loss: 0.9909 - val_acc: 0.8359
Epoch 5/10
6680/6680 [==============================] - 4s 648us/step - loss: 1.0336 - acc: 0.9790 - val_loss: 0.8453 - val_acc: 0.8204
Epoch 6/10
6680/6680 [==============================] - 4s 657us/step - loss: 1.0071 - acc: 0.9855 - val_loss: 0.7548 - val_acc: 0.8491
Epoch 7/10
6680/6680 [==============================] - 4s 667us/step - loss: 0.9907 - acc: 0.9891 - val_loss: 0.7255 - val_acc: 0.8515
Epo

6680/6680 [==============================] - 4s 636us/step - loss: 0.9638 - acc: 0.9946 - val_loss: 0.7093 - val_acc: 0.8491
Epoch 10/10
6680/6680 [==============================] - 4s 643us/step - loss: 0.9567 - acc: 0.9951 - val_loss: 0.7019 - val_acc: 0.8455
Train on 6680 samples, validate on 835 samples
Epoch 1/10
6680/6680 [==============================] - 7s 1ms/step - loss: 2.0987 - acc: 0.6834 - val_loss: 1.9033 - val_acc: 0.8060
Epoch 2/10
6680/6680 [==============================] - 4s 636us/step - loss: 1.2535 - acc: 0.8994 - val_loss: 1.4887 - val_acc: 0.8335
Epoch 3/10
6680/6680 [==============================] - 4s 643us/step - loss: 1.1341 - acc: 0.9436 - val_loss: 1.1769 - val_acc: 0.8479
Epoch 4/10
6680/6680 [==============================] - 4s 639us/step - loss: 1.0734 - acc: 0.9660 - val_loss: 0.9543 - val_acc: 0.8407
Epoch 5/10
6680/6680 [==============================] - 4s 636us/step - loss: 1.0350 - acc: 0.9799 - val_loss: 0.8142 - val_acc: 0.8407
Epoch 6/10
66

6680/6680 [==============================] - 4s 648us/step - loss: 0.9879 - acc: 0.9910 - val_loss: 0.7236 - val_acc: 0.8443
Epoch 8/10
6680/6680 [==============================] - 4s 646us/step - loss: 0.9722 - acc: 0.9936 - val_loss: 0.7239 - val_acc: 0.8551
Epoch 9/10
6680/6680 [==============================] - 4s 643us/step - loss: 0.9608 - acc: 0.9955 - val_loss: 0.7165 - val_acc: 0.8491
Epoch 10/10
6680/6680 [==============================] - 4s 646us/step - loss: 0.9557 - acc: 0.9954 - val_loss: 0.7083 - val_acc: 0.8383
Train on 6680 samples, validate on 835 samples
Epoch 1/10
6680/6680 [==============================] - 7s 1ms/step - loss: 2.0827 - acc: 0.6843 - val_loss: 1.8858 - val_acc: 0.8036
Epoch 2/10
6680/6680 [==============================] - 4s 636us/step - loss: 1.2469 - acc: 0.9040 - val_loss: 1.4921 - val_acc: 0.8359
Epoch 3/10
6680/6680 [==============================] - 4s 648us/step - loss: 1.1346 - acc: 0.9419 - val_loss: 1.1946 - val_acc: 0.8419
Epoch 4/10
66

6680/6680 [==============================] - 4s 660us/step - loss: 1.0346 - acc: 0.9768 - val_loss: 0.8382 - val_acc: 0.8347
Epoch 6/10
6680/6680 [==============================] - 4s 650us/step - loss: 1.0100 - acc: 0.9855 - val_loss: 0.8016 - val_acc: 0.8359
Epoch 7/10
6680/6680 [==============================] - 4s 650us/step - loss: 0.9866 - acc: 0.9897 - val_loss: 0.7417 - val_acc: 0.8419
Epoch 8/10
6680/6680 [==============================] - 4s 648us/step - loss: 0.9741 - acc: 0.9925 - val_loss: 0.7208 - val_acc: 0.8407
Epoch 9/10
6680/6680 [==============================] - 4s 648us/step - loss: 0.9660 - acc: 0.9942 - val_loss: 0.7295 - val_acc: 0.8467
Epoch 10/10
6680/6680 [==============================] - 4s 648us/step - loss: 0.9556 - acc: 0.9954 - val_loss: 0.7068 - val_acc: 0.8407
Train on 6680 samples, validate on 835 samples
Epoch 1/10
6680/6680 [==============================] - 8s 1ms/step - loss: 2.1095 - acc: 0.6740 - val_loss: 1.9099 - val_acc: 0.8060
Epoch 2/10
66

In [118]:
xception_predictions =[]
for feature in test_xception:
    vector = np.array([model.predict(np.expand_dims(feature, axis=0)) for model in Xcept])
    xception_prediction = vector.sum(axis=0)
    xception_predictions.append(np.argmax(xception_prediction))
    



In [126]:
test_accuracy = 100*np.sum(np.array(xception_predictions)==np.argmax(test_targets, axis=1))/len(xception_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 86.6029%


In [120]:
xception_predictions1 = []
for feature in test_xception:
    vector = np.array([model.predict(np.expand_dims(feature, axis=0)) for model in Xcept])
    vector.shape = (-1, vector.shape[2])
    vector1 = np.array([np.argmax(vector[i]) for i in range(len(vector))])
    xception_prediction = np.bincount(vector1)
    xception_predictions1.append(np.argmax(xception_prediction))
    

    

In [125]:
test_accuracy = 100*np.sum(np.array(xception_predictions1)==np.argmax(test_targets, axis=1))/len(xception_predictions1)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 86.6029%


In [123]:
print (np.array(xception_predictions1).shape, np.array(xception_predictions).shape)
sameness = 100*np.sum(np.array(xception_predictions1)== np.array(xception_predictions))/len(xception_predictions1)

(836,) (836,)


In [150]:
print('Sameness: %.4f%%' % sameness)

for i, target in enumerate(xception_predictions):
    if target != xception_predictions1[i]:
        print (i, target, xception_predictions1[i])

Sameness: 98.8038%
23 95 94
34 50 105
40 80 120
44 65 34
52 25 73
72 81 100
75 76 52
93 55 101
98 8 34
110 81 98
120 120 69
123 0 98
128 43 40
134 17 58
146 83 66
148 123 87
158 111 85
159 112 16
164 65 34
168 115 47
175 36 62
181 99 100
190 15 6
192 50 20
202 17 69
222 83 44
255 113 88
258 63 45
271 20 67
287 111 66
306 52 60
334 48 23
341 78 94
346 127 42
348 132 126
351 25 73
368 17 58
383 132 106
390 127 42
395 75 85
414 47 101
419 62 84
423 124 87
442 74 41
443 54 34
445 34 85
447 52 45
452 95 46
475 47 109
485 66 75
493 98 81
496 88 72
500 35 16
501 44 79
504 20 21
516 59 39
532 84 129
534 30 38
537 127 38
552 21 20
566 110 106
571 84 111
573 105 88
575 132 48
600 55 1
604 71 120
628 40 102
659 95 71
662 60 65
692 132 126
698 0 37
728 9 6
742 3 70
745 36 15
756 111 11
780 9 120
783 24 26
815 94 75


In [89]:
ex = np.array([[1,2], [2,3]])

In [35]:
ex

array([[1, 2],
       [2, 3]])

In [36]:
ex.sum(axis =1)

array([3, 5])

In [42]:
xception_model.predict(np.expand_dims(train_xception[1],axis = 0))

array([[  9.93403024e-04,   2.91732344e-04,   7.08184845e-04,
          5.53736056e-04,   8.82767723e-04,   5.65732887e-04,
          5.29449550e-04,   2.93705409e-04,   1.10657529e-04,
          3.76875891e-04,   5.58995525e-04,   4.05269791e-04,
          3.29000788e-04,   3.35938646e-04,   5.50363737e-04,
          1.40384375e-03,   6.56558026e-04,   5.88758790e-04,
          7.13188783e-04,   5.64187067e-04,   1.72838022e-03,
          7.18943193e-04,   5.27972705e-04,   8.41362576e-04,
          2.95340404e-04,   9.37157383e-05,   4.06838109e-04,
          3.13604425e-04,   6.50361064e-04,   5.83247398e-04,
          3.50395625e-04,   5.58093190e-04,   2.92699609e-04,
          2.71333818e-04,   6.56212505e-04,   1.12004590e-03,
          4.38363699e-04,   2.13358202e-04,   1.98076916e-04,
          1.04406138e-03,   3.25224741e-04,   3.92265065e-04,
          6.42737548e-04,   4.45378653e-04,   3.89926689e-04,
          5.67363808e-04,   6.54491305e-04,   4.82334144e-04,
        

In [46]:
train_xception.shape

(6680, 7, 7, 2048)

In [50]:
indexes = np.random.randint(0,high= train_xception.shape[0], size = train_xception.shape[0], dtype = 'int32')

In [51]:
indexes

array([2599, 2323, 3899, ..., 3744,  155, 6589])

In [53]:
train = np.array([train_xception[ind] for ind in indexes])
targ = np.array([train_targets[ind]for ind in indexes])

In [54]:
train.shape

(6680, 7, 7, 2048)

In [55]:
targ.shape

(6680, 133)

In [154]:
np.argmax(train_targets[3])


7